# Knowledge Graph construction and query with extracted software metadata

This notebook first generates a knowledge graph from the information extracted about software repositories. It is later queried to assess the good practices followed by the extracted repositories.

In [1]:
import morph_kgc
import pyoxigraph

## KG Construction
The knowledge graph is generated using Morph-KGC, that uses RML mappings to transform the JSON file into RDF. This tool requires some configuration parameters, where we indicate the desired output serialisation and the name and path to the RML mapping file. Then, the kg is generated and stored as a oxigraph store in the variable `graph`, that it is also saved as a `.nq` file.

In [13]:
config = """
             [CONFIGURATION]
             output_format=N-QUADS
             
             [SOMEF-json]
             mappings=../mappings/mapping-somef-star.ttl
         """

In [14]:
graph = morph_kgc.materialize_oxigraph(config)

INFO | 2023-07-01 10:53:23,541 | 147 mapping rules retrieved.
INFO | 2023-07-01 10:53:23,548 | Mappings processed in 1.014 seconds.
INFO | 2023-07-01 10:53:34,046 | Number of triples generated in total: 16801.


In [18]:
with open('/Users/aiglesias/GitHub/oeg-software-graph/data/somef-kg.nq', 'w') as result:
    result.write(str(graph))

## KG querying - FAIRness assessment

In [50]:
query = """
            PREFIX sd: <https://w3id.org/okn/o/sd#>
            
            SELECT DISTINCT ?p 
            WHERE {
                << ?s ?p ?o >> sd:confidence ?conf
            }
"""

In [51]:
q_res = graph.query(query)

for solution in q_res:
    print(solution['p'])


<https://w3id.org/okn/o/sd#hasSupportScriptLocation>
<https://w3id.org/okn/o/sd#dateModified>
<https://w3id.org/okn/o/sd#author>
<https://w3id.org/okn/o/sd#name>
<https://w3id.org/okn/o/sd#issueTracker>
<https://w3id.org/okn/o/sd#date-created>
<https://w3id.org/okn/o/sd#hasLongName>
<https://w3id.org/okn/o/sd#hasDownloadUrl>
<https://w3id.org/okn/o/sd#hasBuildFile>
<https://w3id.org/okn/o/sd#hasSourceCode>
<https://w3id.org/okn/o/sd#readme>
<https://w3id.org/okn/o/sd#hasDocumentation>
<http://schema.org/license>
<https://w3id.org/okn/o/sd#hasUsageNotes>
<https://w3id.org/okn/o/sd#keywords>
<https://w3id.org/okn/o/sd#hasExecutableInstructions>
<https://w3id.org/okn/o/sd#hasFAQ>
<https://w3id.org/okn/o/sd#contributionGuidelines>
<https://w3id.org/okn/o/sd#softwareRequirements>
<https://w3id.org/okn/o/sd#citation>
<https://w3id.org/okn/o/sd#identifier>
<https://w3id.org/okn/o/sd#contactDetails>
<https://w3id.org/okn/o/sd#hasExample>
<https://w3id.org/okn/o/sd#hasAcknowledgements>
<https:/